# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [5]:
import pandas as pd
import statistics as st
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import numpy as np
from sklearn.preprocessing import scale
from rpy2.robjects import pandas2ri

d:\dev-cpp\anaconda\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('..\\data\\TabellaCompletaProva.xlsx', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**autore_panel** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [3]:
time = table['time'].tolist()
table['time'] = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','autore_panel'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,time,project_depth,remixed,stars_autore,has_avatar_autore,has_bio_autore,followers_autore,tot_loves_autore,tot_views_autore,ranking_autore,panel_stars
time,1.00,-0.00,-0.01,-0.32,-0.15,0.08,-0.28,-0.23,-0.43,-0.69,-0.18
project_depth,-0.00,1.00,-0.42,0.05,0.06,0.03,0.02,0.04,0.05,0.06,0.07
remixed,-0.01,-0.42,1.00,-0.03,-0.03,-0.03,-0.02,-0.03,-0.03,-0.03,-0.17
stars_autore,-0.32,0.05,-0.03,1.00,0.09,-0.04,0.55,0.74,0.85,0.45,0.17
has_avatar_autore,-0.15,0.06,-0.03,0.09,1.00,0.24,0.18,0.15,0.11,0.13,0.08
has_bio_autore,0.08,0.03,-0.03,-0.04,0.24,1.00,0.22,0.12,0.01,-0.08,0.06
followers_autore,-0.28,0.02,-0.02,0.55,0.18,0.22,1.00,0.71,0.63,0.35,0.18
tot_loves_autore,-0.23,0.04,-0.03,0.74,0.15,0.12,0.71,1.00,0.76,0.32,0.19
tot_views_autore,-0.43,0.05,-0.03,0.85,0.11,0.01,0.63,0.76,1.00,0.64,0.12
ranking_autore,-0.69,0.06,-0.03,0.45,0.13,-0.08,0.35,0.32,0.64,1.00,0.16


Come si può osservare dalla matrice, tra la feature **tot_views_autore** e **stars_autore** c'è un elevato grado di correlazione (**0.85**), questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking di un autore. Dato che star_count rappresenta la misura di apprezzamento migliore, si decide di eliminare tot_views_autore. A supporto di questa decisione è la presenza di un alto grado di correlazione tra tot_views_autore e **tot_loves_autore** (**0.76**).

Nonostante il grado di correlazione tra tot_loves_autore e stars_autore sia di **0.74**, esso è comunque nel limite tollerabile. 
Inoltre, questo valore è dovuto dalla presenza di tot_loves_autore nel calcolo del ranking. Di seguito viene mostrata la matrice di correlazione corretta.

In [4]:
table = table.drop(columns = ['tot_views_autore'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,time,project_depth,remixed,stars_autore,has_avatar_autore,has_bio_autore,followers_autore,tot_loves_autore,ranking_autore,panel_stars
time,1.00,-0.00,-0.01,-0.32,-0.15,0.08,-0.28,-0.23,-0.69,-0.18
project_depth,-0.00,1.00,-0.42,0.05,0.06,0.03,0.02,0.04,0.06,0.07
remixed,-0.01,-0.42,1.00,-0.03,-0.03,-0.03,-0.02,-0.03,-0.03,-0.17
stars_autore,-0.32,0.05,-0.03,1.00,0.09,-0.04,0.55,0.74,0.45,0.17
has_avatar_autore,-0.15,0.06,-0.03,0.09,1.00,0.24,0.18,0.15,0.13,0.08
has_bio_autore,0.08,0.03,-0.03,-0.04,0.24,1.00,0.22,0.12,-0.08,0.06
followers_autore,-0.28,0.02,-0.02,0.55,0.18,0.22,1.00,0.71,0.35,0.18
tot_loves_autore,-0.23,0.04,-0.03,0.74,0.15,0.12,0.71,1.00,0.32,0.19
ranking_autore,-0.69,0.06,-0.03,0.45,0.13,-0.08,0.35,0.32,1.00,0.16
panel_stars,-0.18,0.07,-0.17,0.17,0.08,0.06,0.18,0.19,0.16,1.00


# COUNT DATA MODEL

Per costruire i count data model si uttilizza il modulo **rpy2**, il quale permette di poter importare ed utilizzare l'ambiente **R** in Python.

In [6]:
utils = importr('utils')

Installiamo tutte le librerie R necessarie. Questo può essere fatto grazie all'utilizzo del metodo **install_packages()** della libreria R **utils**.

In [ ]:
utils.install_packages('pscl')
utils.install_packages('MASS')
utils.install_packages('stats')
utils.install_packages('lmtest')
utils.install_packages('nonnest2')

Una volta installate si passa all'import delle stesse.

In [7]:
MASS = importr('MASS')
stats = importr('stats')
pscl = importr('pscl')
base = importr('base')
lmtest = importr('lmtest')
nonnest2 = importr('nonnest2')

Il primo passaggio, per la costruzione dei count data model, consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [8]:
featureTable = table
#featureTable['time'] = numTime
logit = featureTable.drop(columns = ['time','project_depth','stars_autore','followers_autore','tot_loves_autore',
                'ranking_autore','id_panel','panel_stars'])
numerical = featureTable.drop(columns = ['id_panel','remixed','has_avatar_autore','has_bio_autore'])

La feature **project_depth** all'interno della tabella, sta ad indicare la profondità di ogni progetto, ovvero il numero di vignette contenute in esso. Se, per una vignetta, è presente quella successiva allora il valore di **extended** relativa ad essa sarà True, altrimenti False. In questa maniera, se un progetto dovesse avere solo una vignetta(ovvero quella del creatore del progetto stesso), la project_depth relativa avrà valore 1. Al fine di ottenere una corretta costruzione del modello di regressione, il valore della project_depth di un progetto non esteso, dovrà necessariamente essere pari a 0, per questo motivo si è deciso di sottrare 1 alle profondità di tutti i progetti.

In [9]:
numerical['project_depth'] = numerical['project_depth'] - 1

bisogna effettuare una log-trasformazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica. Bisogna specificare che, dopo aver effettuato la log-trasformazione, i valori della variabile dipendente **panel_stars** vengono convertiti ad intero. Questo perchè non è possibile avere una variabile dipendente che sia di un tipo diverso da tipo intero.

In [10]:
normTable = np.log1p(numerical)
normTable['panel_stars'] = normTable['panel_stars'].astype(int)
normTable['remixed'] = logit['remixed']
normTable['has_avatar_autore'] = logit['has_avatar_autore']
normTable['has_bio_autore'] = logit['has_bio_autore']

A questo punto si procede con la conversione del dataframe da **pandas DataFrame** ad **R DataFrame**, questa operazione è necessaria affinchè si possa utilizzare il dataFrame. Per effettuare tale conversione si usa il metodo **pandas2ri.py2ri** del modulo rpy2. 

In [12]:
pandas2ri.activate()
rTable = pandas2ri.py2ri(normTable)

lo script successivo permette la costruzione dei count data model, in questo caso si è deciso di costruire 4 modelli, ossia **Poisson**, **Negative Binomial**, **Zero Inflated negative binomial** e **Hurdle Model**, utilizzando rispettivamente la funzione **glm** del package **stats** per il modello di Poisson, la funzione **glm_nb** del package **MASS** per la Negative Binomial, la funzione **zeroinfl** del package **pscl** per la Zero Inflated è la funzione **hurdle** del package **pscl** per Hurdle Model. La variabile **formula** contiene una stringa scritta in notazione **Patsy** utile per poter impostare la variabile dipendente (inserita a sinistra del simbolo tilde ~ ) e le variabili indipendenti che fungeranno da predittori(poste a destra del simbolo tilde). 

In [22]:
formula = """panel_stars ~ project_depth + stars_autore + followers_autore +
             ranking_autore + remixed + has_avatar_autore + has_bio_autore + tot_loves_autore"""


summary = ro.r['summary']

poissonMod = stats.glm(ro.r(formula), family = ro.r('poisson(link = "log")'), data = rTable)
nbMod = MASS.glm_nb(ro.r(formula), data = rTable)
zeroinflMod = pscl.zeroinfl(ro.r(formula), data = rTable,dist = 'negbin')
hurdleMod = pscl.hurdle(ro.r(formula), data = rTable, dist = 'negbin', zero = "binomial")

**MODELLO DI POISSON**

In [23]:
poissonSum = summary(poissonMod)
loglikPoisson = stats.logLik(poissonMod)

poisson = str(poissonSum)
poisson = poisson.split('Deviance')[1]
print(poisson)
print(loglikPoisson)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.5385  -0.8331  -0.6575   0.5990   2.6223  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.96420    0.14144 -13.887  < 2e-16 ***

project_depth         -0.06253    0.02745  -2.278   0.0227 *  

stars_autore           0.20465    0.01270  16.115  < 2e-16 ***

followers_autore      -0.04720    0.01935  -2.439   0.0147 *  

ranking_autore         0.11204    0.01855   6.039 1.55e-09 ***

remixedTRUE           -0.46903    0.03436 -13.650  < 2e-16 ***

has_avatar_autoreTRUE  0.23280    0.10147   2.294   0.0218 *  

has_bio_autoreTRUE     0.05253    0.03255   1.614   0.1065    

tot_loves_autore      -0.09529    0.01644  -5.796 6.80e-09 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for poisson family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10168  on 13739  degrees of fr

**NEGATIVE BINOMIAL**

In [24]:
nbSum = summary(nbMod)
loglikNB = stats.logLik(nbMod)

nb = str(nbSum)
nb = nb.split('Deviance')[1]
print(nb)
print(loglikNB)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.5385  -0.8331  -0.6575   0.5989   2.6221  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.96422    0.14145 -13.886  < 2e-16 ***

project_depth         -0.06253    0.02745  -2.278   0.0227 *  

stars_autore           0.20466    0.01270  16.114  < 2e-16 ***

followers_autore      -0.04720    0.01935  -2.439   0.0147 *  

ranking_autore         0.11204    0.01855   6.039 1.55e-09 ***

remixedTRUE           -0.46903    0.03436 -13.649  < 2e-16 ***

has_avatar_autoreTRUE  0.23280    0.10148   2.294   0.0218 *  

has_bio_autoreTRUE     0.05253    0.03255   1.614   0.1065    

tot_loves_autore      -0.09529    0.01644  -5.796 6.80e-09 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for Negative Binomial(8947.233) family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10168  on 

**ZERO INFLATED NEGATIVE BINOMIAL**

In [25]:
zeroinflSum = summary(zeroinflMod)
loglikZeroinfl = stats.logLik(zeroinflMod)

zeroinfl = str(zeroinflSum)
zeroinfl = zeroinfl.split('Pearson')[1]
print(zeroinfl)
print(loglikZeroinfl)

 residuals:

    Min      1Q  Median      3Q     Max 

-0.9767 -0.6486 -0.2689  0.5985 26.6663 



Count model coefficients (negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.27051    0.16395  -7.749 9.24e-15 ***

project_depth         -0.09019    0.02960  -3.047  0.00231 ** 

stars_autore           0.04978    0.01513   3.290  0.00100 ** 

followers_autore       0.03495    0.02117   1.651  0.09871 .  

ranking_autore         0.11881    0.02110   5.631 1.79e-08 ***

remixedTRUE           -0.42366    0.03679 -11.517  < 2e-16 ***

has_avatar_autoreTRUE  0.21499    0.12161   1.768  0.07707 .  

has_bio_autoreTRUE    -0.01803    0.03544  -0.509  0.61100    

tot_loves_autore      -0.05748    0.01788  -3.214  0.00131 ** 

Log(theta)            15.51054         NA      NA       NA    



Zero-inflation model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)       

**HURDLE MODEL**

In [26]:
hurdleSum = summary(hurdleMod)
loglikHurdle = stats.logLik(hurdleMod)

hurdle = str(hurdleSum)
hurdle = hurdle.split('Pearson')[1]
print(hurdle)
print(loglikHurdle)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.6956 -0.6675 -0.4498  0.7358  4.4032 



Count model coefficients (truncated negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.07545    0.57999  -3.578 0.000346 ***

project_depth         -0.10935    0.09655  -1.133 0.257397    

stars_autore          -0.05868    0.04703  -1.248 0.212106    

followers_autore       0.30377    0.06920   4.390 1.14e-05 ***

ranking_autore         0.09238    0.06539   1.413 0.157767    

remixedTRUE           -0.60261    0.11364  -5.303 1.14e-07 ***

has_avatar_autoreTRUE  0.43985    0.45819   0.960 0.337066    

has_bio_autoreTRUE    -0.14280    0.10553  -1.353 0.176003    

tot_loves_autore      -0.06651    0.06237  -1.066 0.286251    

Log(theta)            16.10066    4.00878   4.016 5.91e-05 ***

Zero hurdle model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)  

**LIKELIHOOD RATIO TEST E TEST DI VUONG**

**Likelihood ratio test**

In [27]:
lrt = lmtest.lrtest(poissonMod,nbMod)
print(lrt)

Likelihood ratio test



Model 1: panel_stars ~ project_depth + stars_autore + followers_autore + 

    ranking_autore + remixed + has_avatar_autore + has_bio_autore + 

    tot_loves_autore

Model 2: panel_stars ~ project_depth + stars_autore + followers_autore + 

    ranking_autore + remixed + has_avatar_autore + has_bio_autore + 

    tot_loves_autore

  #Df LogLik Df Chisq Pr(>Chisq)

1   9 -10211                    

2  10 -10211  1 0.178     0.6731



**Test di Vuong**

In [19]:
nb_hurdle_vuong = nonnest2.vuongtest(hurdleMod,nbMod)
print(nb_hurdle_vuong)



Model 1 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.066,   p = 2.74e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 18.326,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 18.326,   p = 1



In [28]:
zero_hurdle_test = nonnest2.vuongtest(zeroinflMod, hurdleMod)
print(zero_hurdle_test)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.106,   p = 1.77e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = -3.983,   p = 1

  H1B: Model 2 fits better than Model 1 

    z = -3.983,   p = 3.402e-05



In [21]:
nb_zeroinfl_vuong = nonnest2.vuongtest(zeroinflMod,nbMod)
print(nb_zeroinfl_vuong)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.042,   p = <2e-16



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 16.069,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 16.069,   p = 1

